In [ ]:
import astropy.units as u
from astropy.constants import G
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline

import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

In [ ]:
tbl = np.genfromtxt('../data/MW-Menc.txt', names=True, delimiter=',')

In [ ]:
pot = gp.CCompositePotential()

pot['bar'] = gp.LongMuraliBarPotential(m=1E10*u.Msun, 
                                       a=4*u.kpc, b=0.1*u.kpc, c=0.4*u.kpc, 
                                       units=galactic)
pot['sph'] = gp.HernquistPotential(m=5E9, c=0.5, 
                                   units=galactic)
pot_bar_bulge = pot.copy()

pot['disk'] = gp.MiyamotoNagaiPotential(m=2E10*u.Msun, a=3*u.kpc, b=280*u.pc, 
                                        units=galactic)
pot_no_halo = pot.copy()

pot['halo'] = gp.NFWPotential(m=5E11, r_s=18*u.kpc, 
                              units=galactic)

In [ ]:
grid = np.linspace(-15, 15, 256)

fig,axes = plt.subplots(1, 2, figsize=(12,6))
_ = pot.plot_density_contours(grid=(grid, grid, 0), ax=axes[0])
_ = pot.plot_density_contours(grid=(grid, 0, grid), ax=axes[1])

In [ ]:
pot.mass_enclosed([20, 0,0.])

In [ ]:
xyz = np.zeros((3, 256))
xyz[0] = np.logspace(-1, 2.5, xyz.shape[1])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,6))

axes[0].loglog(xyz[0], pot.mass_enclosed(xyz).value, marker='', lw=2.)
axes[0].set_xlim(0.1, 250.)
axes[0].tick_params(which='both', right=True, top=True)
axes[0].errorbar(tbl['r'], tbl['Menc'], yerr=(tbl['Menc_neg_err'], tbl['Menc_pos_err']),
                 marker='o', ls='none')

vcirc = np.sqrt(G*pot.mass_enclosed(xyz)/(xyz[0]*u.kpc)).to(u.km/u.s)
axes[1].plot(xyz[0], vcirc, marker='', lw=2.)
axes[1].axhline(220.)
axes[1].axvline(8.)
axes[1].fill_between([0,20], 200., 240, alpha=0.4, color='#888888')
axes[1].set_xlim(0, 20)
axes[1].set_ylim(0, 280)
axes[1].set_xlabel("$r$ [kpc]")
axes[1].set_ylabel(r"$v_c$ [${\rm km}\,{\rm s}^{-1}$]")

fig.tight_layout()

### Compare surface density to measurements

In [ ]:
from scipy.integrate import quad

In [ ]:
def func(z, R, pot, axis=0):
    q = np.array([0, 0, z])
    q[axis] = R
    return pot.density(q).value[0]

Total surface mass density in stars from [Bovy 2012](http://iopscience.iop.org/article/10.1088/0004-637X/751/2/131/pdf)

In [ ]:
bovy_R = 8.*u.kpc
bovy_sdens = 30. * u.Msun/u.pc**2 # +/- 1

In [ ]:
Rs = np.linspace(4, 10, 16)
surf_dens = np.zeros_like(Rs)

for i,R in enumerate(Rs):
    _surf_dens,_ = quad(func, -1.1, 1.1, args=(R, pot_no_halo))
    surf_dens[i] = (_surf_dens*u.Msun/u.kpc**2).to(u.Msun/u.pc**2).value

In [ ]:
plt.figure(figsize=(5,5))
plt.semilogy(Rs, surf_dens, marker='', linestyle='-')
plt.scatter(bovy_R, bovy_sdens)
plt.xlim(4, 10)
plt.ylim(10, 1000)
plt.xlabel(r"$R$ [kpc]")
plt.ylabel(r"$\Sigma(R)$ [${\rm M}_\odot \, {\rm pc}^{-2}$]")

Surface mass densities read off of Fig. 3 in [Portail 2016](https://arxiv.org/pdf/1608.07954v2.pdf)

In [ ]:
R = np.linspace(0, 10, 128)
Rc = 2
plt.loglog(R, (R**-1)*np.exp(-R/Rc))

In [ ]:
pot = gp.CCompositePotential()

pot['bar'] = gp.LongMuraliBarPotential(m=1E10*u.Msun, 
                                       a=4*u.kpc, b=0.5*u.kpc, c=0.5*u.kpc, 
                                       units=galactic)
pot['inner_disk'] = gp.MiyamotoNagaiPotential(m=5E9*u.Msun, 
                                              a=0.5*u.kpc, b=280*u.pc, 
                                              units=galactic)

pot['sph'] = gp.JaffePotential(m=1E9, c=4, 
                               units=galactic)

pot_bar_bulge = pot.copy()

pot['disk'] = gp.MiyamotoNagaiPotential(m=2E10*u.Msun, a=3*u.kpc, b=280*u.pc, 
                                        units=galactic)
pot_no_halo = pot.copy()

pot['halo'] = gp.NFWPotential(m=5E11, r_s=18*u.kpc, 
                              units=galactic)

grid = np.linspace(-8, 8, 256)

l_min = np.log10(pot.density([0.1, 0, 0]).value[0])
l_max = np.log10(pot.density([20., 0, 0]).value[0])

fig,axes = plt.subplots(1, 2, figsize=(12,6))
_ = pot.plot_density_contours(grid=(grid, grid, 0), ax=axes[0], 
                              levels=sorted(np.logspace(l_min, l_max, 16)),
                              cmap='viridis')
_ = pot.plot_density_contours(grid=(grid, 0, grid), ax=axes[1],
                              levels=sorted(np.logspace(l_min, l_max, 16)),
                              cmap='viridis')

In [ ]:
xy_Rs = np.linspace(1E-2, 8, 16)
surf_dens_x = np.zeros_like(Rs)
surf_dens_y = np.zeros_like(Rs)

for i,R in enumerate(xy_Rs):
    _surf_dens,_ = quad(func, -1.1, 1.1, args=(R, pot_bar_bulge, 0))
    surf_dens_x[i] = (_surf_dens*u.Msun/u.kpc**2).to(u.Msun/u.pc**2).value
    
    _surf_dens,_ = quad(func, -1.1, 1.1, args=(R, pot_bar_bulge, 1))
    surf_dens_y[i] = (_surf_dens*u.Msun/u.kpc**2).to(u.Msun/u.pc**2).value

These points were read off the plot for just the bar+bulge components

In [ ]:
x = [0, 1, 2, 3, 4, 6] * u.kpc
x_sdens = ([4E9, 2.5E9, 8E8, 6.5E8, 4E8, 8E6] * u.Msun/u.kpc**2).to(u.Msun/u.pc**2)

y = [0, 1, 2, 3, 4, 6] * u.kpc
y_sdens = ([4E9, 6E8, 8E7, 6.5E7, 6.5E7, 5E6] * u.Msun/u.kpc**2).to(u.Msun/u.pc**2)

plt.figure(figsize=(7,5))

plt.semilogy(xy_Rs, surf_dens_x, marker='', ls='-', color='r')
plt.semilogy(-xy_Rs, surf_dens_x, marker='', ls='-', color='r')
plt.semilogy(xy_Rs, surf_dens_y, marker='', ls='-', color='b')
plt.semilogy(-xy_Rs, surf_dens_y, marker='', ls='-', color='b')

plt.semilogy(x, x_sdens, marker='o', color='r', ls='none', mec='k', mew=1.)
plt.semilogy(y, y_sdens, marker='o', color='b', ls='none', mec='k', mew=1.)
plt.semilogy(-x, x_sdens, marker='o', color='r', ls='none', mec='k', mew=1.)
plt.semilogy(-y, y_sdens, marker='o', color='b', ls='none', mec='k', mew=1.)

plt.xlim(-8.1, 8.1)
plt.ylim(1E1, 1E4)
plt.xlabel(r"$x,y$ [kpc]")
plt.ylabel(r"$\Sigma(R)$ [${\rm M}_\odot \, {\rm pc}^{-2}$]")

These points were read off the plot for just the total baryonic

In [ ]:
x = [0, 2, 4, 6, 8] * u.kpc
x_sdens = ([4E9, 1E9, 5.5E8, 1E8, 4E7] * u.Msun/u.kpc**2).to(u.Msun/u.pc**2)

y = [0, 1, 2, 4, 6, 8] * u.kpc
y_sdens = ([4E9, 8E8, 2.2E8, 2E8, 1E8, 4E7] * u.Msun/u.kpc**2).to(u.Msun/u.pc**2)

plt.figure(figsize=(7,5))

plt.semilogy(xy_Rs, surf_dens_x, marker='', ls='-', color='r')
plt.semilogy(-xy_Rs, surf_dens_x, marker='', ls='-', color='r')
plt.semilogy(xy_Rs, surf_dens_y, marker='', ls='-', color='b')
plt.semilogy(-xy_Rs, surf_dens_y, marker='', ls='-', color='b')

plt.semilogy(x, x_sdens, marker='o', color='r', ls='none', mec='k', mew=1.)
plt.semilogy(y, y_sdens, marker='o', color='b', ls='none', mec='k', mew=1.)
plt.semilogy(-x, x_sdens, marker='o', color='r', ls='none', mec='k', mew=1.)
plt.semilogy(-y, y_sdens, marker='o', color='b', ls='none', mec='k', mew=1.)

plt.xlim(-8.1, 8.1)
plt.ylim(1E1, 1E4)
plt.xlabel(r"$x,y$ [kpc]")
plt.ylabel(r"$\Sigma(R)$ [${\rm M}_\odot \, {\rm pc}^{-2}$]")